In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict

"""
1. ĐỊNH NGHĨA MÔ HÌNH MẠNG NEURAL
Kiến trúc mô hình sẽ được sử dụng trên cả server và các client.
"""
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        # Mạng fully-connected với:
        # - Input: 10 features
        # - Output: 2 classes 
        self.fc = nn.Linear(10, 2)
        
    def forward(self, x):
        # Phương thức forward pass của mô hình
        return self.fc(x)

"""
2. HÀM TẠO DỮ LIỆU GIẢ LẬP CHO CÁC CLIENT
Giả lập dữ liệu cho client.
"""
def get_client_data(num_clients):
    client_data = []
    for i in range(num_clients):
        # Mỗi client có:
        # - 100 samples (dòng dữ liệu)
        # - 10 features (đặc trưng)
        data = torch.randn(100, 10) * (i+1)  # Scale khác nhau cho mỗi client
        
        # Nhãn phân loại ngẫu nhiên (0 hoặc 1)
        labels = torch.randint(0, 2, (100,))
        
        client_data.append((data, labels))
    
    # Kết quả trả về là list các tuple (dữ liệu, nhãn) cho từng client
    return client_data

"""
3. HÀM HUẤN LUYỆN TRÊN TỪNG CLIENT
Huấn luyện cục bộ trên từng client.
Mỗi client sẽ nhận bản sao mô hình từ server và cập nhật trên dữ liệu cục bộ của mình.
"""
def client_update(model, data, labels, epochs=5, lr=0.01):
    # Định nghĩa hàm loss (CrossEntropy cho bài toán phân loại)
    criterion = nn.CrossEntropyLoss()
    
    # Sử dụng Stochastic Gradient Descent (SGD) để cập nhật trọng số
    # model.parameters(): lấy tất cả các trọng số có thể huấn luyện của mô hình
    # lr=lr: learning rate - tốc độ học
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    # Chuyển mô hình sang chế độ huấn luyện
    model.train()
    
    # Vòng lặp huấn luyện
    for epoch in range(epochs):
        # Reset gradients về 0 trước mỗi bước cập nhật 
        optimizer.zero_grad()
        
        # Forward pass: tính toán đầu ra mô hình
        outputs = model(data)
        
        # Tính toán loss giữa đầu ra và nhãn thực
        loss = criterion(outputs, labels)
        
        # Backward pass: tính toán gradients của loss theo tất cả các trọng số
        loss.backward()
        
        # Cập nhật trọng số: weight = weight - lr * gradient
        optimizer.step()
    
    # Trả về state_dict chứa các trọng số đã cập nhật
    return model.state_dict()

"""
4. HÀM TỔNG HỢP TRỌNG SỐ (FEDERATED AVERAGING)
Server nhận trọng số từ các client và tính trung bình để cập nhật mô hình toàn cục.
"""
def federated_averaging(server_model, client_weights):
    # OrderedDict để giữ nguyên thứ tự các trọng số
    averaged_weights = OrderedDict()
    
    # Duyệt qua từng layer trong mô hình
    for key in server_model.state_dict().keys():
        # Tạo stack chứa các trọng số tương ứng từ tất cả client
        weights_stack = torch.stack([weights[key] for weights in client_weights])
        
        # Tính trung bình các trọng số
        averaged_weights[key] = torch.mean(weights_stack, dim=0)
    
    # Cập nhật trọng số mới cho server model
    server_model.load_state_dict(averaged_weights)
    
    return server_model

"""
5. QUY TRÌNH FEDERATED LEARNING
- num_clients: số lượng client tham gia
- rounds: số vòng huấn luyện
"""
def federated_learning(num_clients=5, rounds=10):
    # Khởi tạo mô hình trên server
    server_model = SimpleModel()
    print("Server model initialized")
    
    # Giả lập dữ liệu trên các client
    client_data = get_client_data(num_clients)
    print(f"Generated data for {num_clients} clients")
    
    # Vòng lặp huấn luyện chính
    for round in range(rounds):
        print(f"\n=== Round {round + 1}/{rounds} ===")
        client_weights = []  # Lưu trữ trọng số từ các client
        
        # Huấn luyện trên từng client
        for i in range(num_clients):
            # Tạo bản sao mô hình server cho client hiện tại
            client_model = SimpleModel()
            client_model.load_state_dict(server_model.state_dict()) # load_state_dict sao chép chính xác các trọng số từ server
            
            # Lấy dữ liệu của client hiện tại
            data, labels = client_data[i]
            
            # Huấn luyện cục bộ và nhận lại trọng số đã cập nhật
            weights = client_update(client_model, data, labels)
            client_weights.append(weights)
            
            print(f"Client {i} completed local training")
        
        # Tổng hợp trọng số từ các client
        server_model = federated_averaging(server_model, client_weights)
        

        # Đánh giá mô hình 
        with torch.no_grad(): # torch.no_grad(): tắt tính gradient để tiết kiệm bộ nhớ
            server_model.eval() # eval(): chuyển mô hình sang chế độ đánh giá
            # Lấy dữ liệu từ client 0 làm ví dụ đánh giá
            test_data, test_labels = client_data[0] # Tính accuracy trên dữ liệu của client 0 
            outputs = server_model(test_data)
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == test_labels).sum().item() / test_labels.size(0)
            print(f"Server model accuracy: {accuracy:.2f}")
    
    # Trả về mô hình cuối cùng sau khi huấn luyện
    return server_model

# Chạy thử Federated Learning
if __name__ == "__main__":
    print("Starting Federated Learning...")
    final_model = federated_learning(num_clients=5, rounds=10)
    print("\nFederated Learning completed!")
    print("Final model architecture:")
    print(final_model)

Starting Federated Learning...
Server model initialized
Generated data for 5 clients

=== Round 1/10 ===
Client 0 completed local training
Client 1 completed local training
Client 2 completed local training
Client 3 completed local training
Client 4 completed local training
Server model accuracy: 0.48

=== Round 2/10 ===
Client 0 completed local training
Client 1 completed local training
Client 2 completed local training
Client 3 completed local training
Client 4 completed local training
Server model accuracy: 0.49

=== Round 3/10 ===
Client 0 completed local training
Client 1 completed local training
Client 2 completed local training
Client 3 completed local training
Client 4 completed local training
Server model accuracy: 0.47

=== Round 4/10 ===
Client 0 completed local training
Client 1 completed local training
Client 2 completed local training
Client 3 completed local training
Client 4 completed local training
Server model accuracy: 0.48

=== Round 5/10 ===
Client 0 completed loca